In [1]:
from pymongo import MongoClient
import pprint

import pandas as pd
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import requests
from bs4 import BeautifulSoup

import json
import time
import copy

import os

from time import sleep
from selenium import webdriver

import sys

Step 1:
    gather data for three more hikes
    loop through data
    create dataframes
    join df's for 4 and 5 stars
    perform z-test

Step 2:
    perform further statistical tests
    gather more hike data?
    gather data on individual users and perform further tests

In [25]:
# Collecting hiking trail links from https://www.alltrails.com/explore in a particular location
hrefs = []
area_link = 'http://deertier.com/Leaderboard/AnyPercentRealTime'
r = requests.get(area_link)
# find_all('a', itemprop="url" ... ('href')

links = []
for i, href in enumerate(hrefs):
    cleaned_href = ('/').join(href.split('/')[2:])
    links.append('https://www.alltrails.com/' + cleaned_href)



In [26]:
links

['https://www.alltrails.com/trail/us/new-jersey/long-path-gw-bridge-to-lamont-doherty-earth-observatory?ref=result-card']

In [9]:
# instantiating variables to be columns in dataframe
written_review = ""
ratings = []
dates = []
types = []
review_text = []
reviewer_links = []

# Loop through links to get html, save to mongo, extract relevant data
for link in links:
    driver = webdriver.Chrome()
    driver.get(link)
    sleep(3)

    driver.execute_script("window.scrollTo(0, 10000)")

# find 'show more reviews button'
    search = driver.find_element_by_css_selector("#reviews > div.styles-module__container___px-t2.xlate-none > button")
    
#prevents error being thrown once total trail reviews is reached; 
    showing_results = driver.find_element_by_css_selector('#reviews > div.styles-module__container___px-t2.xlate-none > div')
    total_ = int(showing_results.text.split(' ')[-1])
    per_page = int(showing_results.text.split(' ')[-3])

# clicks through to see additional reviews
    for _ in range((total_//per_page)-1):
        search.click()
        sleep(1)

# get page html
    html = driver.page_source
    time.sleep(2)

# add html to mongo
    client = MongoClient()
    db = client.alltrails
    trail_name = (link.split('/'))[-1]
    pages = db.trail_name
    pages.insert_one({'link': link ,'html': html})
    
# parsing data
    soup = BeautifulSoup(html, 'html.parser')

# getting trail title, description, difficulty, info
    soup.find('h1', class_='xlate-none styles-module__name___1nEtW').text.rstrip()
    soup.find('p', id="auto-overview").text.rstrip()
    soup.find('span', class_="styles-module__diff___22Qtv styles-module__moderate___3w1it styles-module__selected___3fawg").text.rstrip()
    soup.find('span', class_="styles-module__detailData___kQ-eK").text.rstrip()

# getting reviewer data for trail, appending to lists
    for i, review in enumerate(soup.find_all('div', itemprop="review")):

        ratings.append(soup.find_all('span', class_="MuiRating-root default-module__rating___1k45X MuiRating-sizeLarge MuiRating-readOnly")[i]['aria-label'])
        dates.append(soup.find_all('span', class_="styles-module__dateTrailDetails___3qgZC xlate-none")[i].text.rstrip())
        types.append(soup.find_all('span', class_="styles-module__tag___2s-oD styles-module__activityTag___3-RdN")[i].text.rstrip())

        written_review = soup.find_all('div', class_="styles-module__container___3etfA")[i].find('p', itemprop="reviewBody")

        if written_review == None:
            review_text.append(None)

        else:
            review_text.append(soup.find_all('div', class_="styles-module__container___3etfA")[i].find('p', itemprop="reviewBody").text.rstrip())        

        reviewer_links.append('alltrails.com' + soup.find_all('div', class_="styles-module__container___3etfA")[1].find('a', class_="styles-module__link___2i6Za styles-module__recording___3jQX3 xlate-none")['href'])
    
    # for trail in find('ul', class_=" float-delete-button").find_all('li', class_="styles-module__container___10uYZ"):

# close Selenium ChromeDriver
#     driver.close()

In [11]:
# collecting data into dataframe
df = pd.DataFrame({'rating': ratings, 'date':dates, 'desc': types, 'reviews': review_text, 'links': reviewer_links}).sort_values('rating')

,rating,date,desc,reviews,links
102,1 Star,"June 16, 2020",Hiking,None,alltrails.com/explore/recording/long-path-gw-b...
4,1 Star,"November 28, 2020",Running,None,alltrails.com/explore/recording/long-path-gw-b...
208,1 Star,"July 18, 2017",Hiking,None,alltrails.com/explore/recording/long-path-gw-b...
88,1 Star,"August 14, 2020",Hiking,None,alltrails.com/explore/recording/long-path-gw-b...
21,1 Star,"June 25, 2020",Hiking,Ross Dock trail start to Englewood and back,alltrails.com/explore/recording/long-path-gw-b...
94,1 Star,"July 19, 2020",Hiking,None,alltrails.com/explore/recording/long-path-gw-b...
202,2 Stars,"January 7, 2018",Hiking,None,alltrails.com/explore/recording/long-path-gw-b...
178,2 Stars,"April 14, 2019",Hiking,None,alltrails.com/explore/recording/long-path-gw-b...
104,2 Stars,"June 13, 2020",Hiking,None,alltrails.com/explore/recording/long-path-gw-b...
37,2 Stars,"June 21, 2018",Hiking,it barely splits off from the road. it's hard ...,alltrails.com/explore/recording/long-path-gw-b...


### Frequentist Approach

In [48]:
five = df[df['rating'] == '5 Stars']['reviews'].count() #num of written reviews left by five-star reviewers
four = df[df['rating'] == '4 Stars']['reviews'].count() #num of written reviews left by four-star reviewers
num_five = df[df['rating']=='5 Stars']['rating'].count() #num of five-star reviewers
num_four = df[df['rating']=='4 Stars']['rating'].count() #num of four-star reviewers

table = pd.DataFrame({'Written Reviews': [four, five], 'Total Reviewers': [num_four, num_five]}, index = ['4 Stars', '5 Stars']).T
table = table.style.set_properties(**{'text-align': 'center'})
table

,4 Stars,5 Stars
Written Reviews,15,9
Total Reviewers,46,73


At first glance it is obvious that a much higher percentage of 4 star reviewers left written reviews than 5 star reviewers.

### Bayseian Approach